In [39]:
import sys
import os
import glob
import pandas as pd
import soundfile as sf
import numpy as np
import pickle
from sklearn import preprocessing
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import librosa
import torch
from torch.utils.data import DataLoader
from Datasets.SimpleDataset import SimpleDataset
import torch.nn.functional as F
from torch.utils.data import random_split
import torch.nn as nn
import csv

c:\Users\Thats\miniconda3\envs\ani\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create a dictionary containing file path and its label ##

In [40]:
file_to_label_dict = {}

In [28]:
filename = "Data\\archive(1)\\birdsong_metadata.csv"
base_directory = "Data\\archive(1)\\songs\\songs\\wav"

with open (filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)

    fields = next(csvreader)
    fields_idx_dict = dict(zip(fields, range(len(fields))))
    
    for row in csvreader:
        file_id = row[fields_idx_dict["file_id"]]
        label = row[fields_idx_dict["english_cname"]]

        sound_filepath = os.path.join(base_directory, "xc" + file_id + ".wav")
        file_to_label_dict[sound_filepath] = label
        


In [29]:
import pickle

with open('pickles/filelabeldictbrit.pkl', 'wb') as fp:
    pickle.dump(file_to_label_dict, fp)
    print("Pickle Successful")

Pickle Successful


Load pickle

In [41]:
with open('pickles/filelabeldictbrit.pkl', 'rb') as fp:
    file_to_label_dict = pickle.load(fp)
    print("Pickle loaded")

Pickle loaded


In [42]:
print(file_to_label_dict)

{'Data\\archive(1)\\songs\\songs\\wav\\xc132608.wav': 'Common Redpoll', 'Data\\archive(1)\\songs\\songs\\wav\\xc132611.wav': 'Common Redpoll', 'Data\\archive(1)\\songs\\songs\\wav\\xc35068.wav': 'Common Redpoll', 'Data\\archive(1)\\songs\\songs\\wav\\xc82715.wav': 'Marsh Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc64685.wav': 'Marsh Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc64686.wav': 'Marsh Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc120664.wav': 'Sedge Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc120666.wav': 'Sedge Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc76767.wav': 'Sedge Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc124052.wav': 'Eurasian Reed Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc124053.wav': 'Eurasian Reed Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc27999.wav': 'Eurasian Reed Warbler', 'Data\\archive(1)\\songs\\songs\\wav\\xc113082.wav': 'Long-tailed Tit', 'Data\\archive(1)\\songs\\songs\\wav\\xc101371.wav': 'Long-tailed 

# Create dataset from file label dict #

### Convert labels into class indices ###

In [43]:
lables_int = None

In [44]:
labels = list(file_to_label_dict.values())
le = preprocessing.LabelEncoder()
le.fit(labels)
labels_int = le.transform(labels)

keys = list(file_to_label_dict.keys())

file_to_class_int = dict(zip(keys, labels_int))

Pickle Labels_int

In [34]:
with open('./pickles/labels_int_brit.pkl', 'wb') as fp:
    pickle.dump(labels_int, fp)
    print("Pickle Successful")

Pickle Successful


In [35]:
with open('pickles/labels_int_brit.pkl', 'rb') as fp:
    labels_int = pickle.load(fp)
    print("Pickle loaded")

Pickle loaded


In [3]:
one_hot_labels = None

In [27]:
num_classes = np.max(labels_int)
label_binarizer = preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(labels_int)+1))
one_hot_labels = label_binarizer.transform(labels_int)

[ 3 20 20 ... 57 57 57]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]


Pickling of one hots

In [28]:
with open('./pickles/one_hot.pkl', 'wb') as fp:
    pickle.dump(one_hot_labels, fp)
    print("Pickle Successful")

Pickle Successful


In [4]:
with open('pickles/one_hot.pkl', 'rb') as fp:
    one_hot_labels = pickle.load(fp)
    print("Pickle loaded")

Pickle loaded


## Convert audio files to a simple numerical representation for the baseline ##

In [ ]:
baseline_features = []

In [ ]:
from torchaudio import load

for file in keys:
# audio file is decoded on the fly
    

## Convert audio files into wav2vec embeddings ##

In [36]:
features = []

In [37]:
'''
Modified from https://bagustris.wordpress.com/2022/08/23/acoustic-feature-extraction-with-transformers/
'''

from transformers import Wav2Vec2Processor, Wav2Vec2Model
import librosa
import torch

# load model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

for file in keys:
# audio file is decoded on the fly
    print(file)
    array, fs = librosa.load(file, sr=16000)
    # print(len(array))
    input = processor(array.squeeze(), sampling_rate=fs, return_tensors="pt")

    # apply the model to the input array from wav
    with torch.no_grad():
        outputs = model(**input)

    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).numpy()
    print(f"Hidden state shape: {last_hidden_states.shape}")
    features.append(last_hidden_states)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data\archive(1)\songs\songs\wav\xc132608.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc132611.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc35068.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc82715.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc64685.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc64686.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc120664.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc120666.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc76767.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc124052.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc124053.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc27999.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc113082.wav
Hidden state shape: (768,)
Data\archive(1)\songs\songs\wav\xc101371.wav
Hidden state shape: (768,

Pickle the datapoint list

In [38]:
with open('./pickles/datapoints_brit.pkl', 'wb') as fp:
    pickle.dump(features, fp)
    print("Pickle Successful")

Pickle Successful


Load datapoint list

In [39]:
with open('pickles/datapoints_brit.pkl', 'rb') as fp:
    features = pickle.load(fp)
    print("Pickle loaded")

features = np.array(features)
features = torch.from_numpy(features).unsqueeze(1)
print(features.shape)

Pickle loaded
torch.Size([264, 1, 768])


Testing that datset creates properly

In [78]:
# features = np.array(features)
# features = torch.from_numpy(features).unsqueeze(1)
# print(features.shape)
test = SimpleDataset(features, one_hot_labels)
test_loader = DataLoader(test, batch_size=5, shuffle=True)

data, label = next(iter(test_loader))
print(data.shape)

torch.Size([5, 1, 768])


## Create Neural Network ##

In [54]:
# import sys
# import os
# import glob
# import pandas as pd
# import soundfile as sf
import numpy as np
import pickle
# from sklearn import preprocessing
# from transformers import Wav2Vec2Processor, Wav2Vec2Model
# import librosa
import torch
from torch.utils.data import DataLoader
from Datasets.SimpleDataset import SimpleDataset
# import torch.nn.functional as F
from torch.utils.data import random_split
import torch.nn as nn

In [55]:
labels_int = None
features = None

with open('pickles/labels_int_brit.pkl', 'rb') as fp:
    labels_int = pickle.load(fp)
    print("Pickle loaded")

with open('pickles/datapoints_brit.pkl', 'rb') as fp:
    features = pickle.load(fp)
    print("Pickle loaded")

features = np.array(features)
features = np.pad(features, ((0, 0), (0, 16)))
features = np.array(np.split(features, 28, axis=1))
features = features.reshape((features.shape[1], features.shape[0], -1))
print(features.shape)
features = torch.from_numpy(features).unsqueeze(1)
features = features.repeat(1, 3, 1, 1)
print(features.shape)

Pickle loaded
Pickle loaded
(264, 28, 28)
torch.Size([264, 3, 28, 28])


In [56]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda


In [57]:
from resnet1d.resnet1d import ResNet1D
from model.test_net import CNN
from model.Mod_Resnet import Mod_Resnet

# model = ResNet1D(1, 2, 3, 1, 1, 8, 59).to(device)
model = Mod_Resnet().to(device)
model.fc = nn.Linear(512, 88) # assuming that the fc7 layer has 512 neurons, otherwise change it 
model.cuda()
print(model)

Using cache found in C:\Users\Thats/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Thats\miniconda3\envs\ani\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Thats\miniconda3\envs\ani\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Mod_Resnet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [58]:
# print(type(labels_int))
bird_call_dataset = SimpleDataset(features, labels_int)
train_size = int(0.6*len(bird_call_dataset))
test_size = len(bird_call_dataset) - train_size
train_set, test_set = random_split(bird_call_dataset, [train_size, test_size])

train_dataloader = DataLoader(train_set, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=4, shuffle=True)

In [59]:
loss_fn = nn.CrossEntropyLoss()
# Try different learning rate
learning_rate = 0.01

# Try different optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [60]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y.type(torch.LongTensor)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        # print(X.shape)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [63]:
total_loss = []
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            y = y.type(torch.LongTensor)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    total_loss.append(test_loss)
    correct /= size
    print(f"Valid Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [64]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4.007364  [    0/  158]
Valid Error: 
 Accuracy: 0.9%, Avg loss: 9.144879 

Epoch 2
-------------------------------
loss: 6.350011  [    0/  158]
Valid Error: 
 Accuracy: 0.9%, Avg loss: 6.195156 

Epoch 3
-------------------------------
loss: 4.596655  [    0/  158]
Valid Error: 
 Accuracy: 0.0%, Avg loss: 15.631443 

Epoch 4
-------------------------------
loss: 4.248558  [    0/  158]
Valid Error: 
 Accuracy: 0.9%, Avg loss: 12.164519 

Epoch 5
-------------------------------
loss: 4.334021  [    0/  158]
Valid Error: 
 Accuracy: 0.9%, Avg loss: 6.671244 

Epoch 6
-------------------------------
loss: 4.078135  [    0/  158]
Valid Error: 
 Accuracy: 0.0%, Avg loss: 5.258235 

Epoch 7
-------------------------------
loss: 4.883547  [    0/  158]
Valid Error: 
 Accuracy: 0.0%, Avg loss: 7.712461 

Epoch 8
-------------------------------
loss: 4.142620  [    0/  158]
Valid Error: 
 Accuracy: 0.0%, Avg loss: 9.353309 

Epoch 9
--------------

In [66]:
import matplotlib.pyplot as plt

num_epochs = range(100)

plt.plot(num_epochs, total_loss)
plt.ylabel("Loss")
plt.xlabel("Epochs")

AttributeError: module 'matplotlib' has no attribute 'plot'

## Baseline - Using the raw audio files ##

In [47]:
from torchaudio import load

features = []
max_dim = 0
for file in keys:
# audio file is decoded on the fly
    # print(file)
    array, fs = load(file)
    features.append(array)
    if (array.shape[1] > max_dim):
        max_dim = array.shape[1]

print(max_dim)

11121127
